# Инициализация

In [2]:
%%capture 

!pip install geopandas
!pip uninstall -y rtree
!sudo apt install libspatialindex-dev
!pip install rtree

In [20]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import json
import re

from shapely.geometry import Point, shape
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [4]:
import sys

from google.colab import drive
drive.mount('/content/drive')

ROOT = "/content/drive/MyDrive/projects-ds/birds/"
sys.path.append(os.path.join(ROOT, 'code'))

Mounted at /content/drive


In [5]:
from bird_logger import Logger, LogOperation
from bird_progress import ProgressBox
from bird_files import Bird

# Подготовка файла областей и районов

In [6]:
oblast = gpd.read_file(os.path.join(ROOT, "joins/admin_level_4.csv"), encoding="utf-8")
oblast = oblast.loc[:,["geometry","name"]].rename(columns={"name":"oblast"})
oblast = oblast.drop(oblast[oblast.oblast.isin(["Автономна Республіка Крим", "Сумска"])].index)
oblast = oblast.drop(oblast[oblast.oblast=="Севастополь"].index[0])
oblast.oblast = oblast.oblast.str.replace("ё", 'е')

raion = gpd.read_file(os.path.join(ROOT, "joins/admin_level_6.csv"), encoding="utf-8")
raion = raion.loc[:,["geometry","name"]].rename(columns={"name":"raion"})
drops = ["Тарко-Сале", "Беляевка", "Армянский городской совет", "Сакский городской совет", "Красноперекопский городской совет", 
            "Керченский городской совет", "Алуштинский городской совет", "Евпаторийский городской совет", 
            "Ялтинский городской совет", "Феодосийский городской совет", "Джанкойский городской совет", 
            "Судакский городской совет", "Симферопольский городской совет"]
raion = raion.drop(raion[raion.raion.isin(drops)].index)
raion.raion = raion.raion.str.replace("ё", 'е')

assert raion.loc[606].raion == "городской округ Ирбит"
assert raion.loc[2150].raion == "ЗАТО Углегорск"
raion = raion.drop([606, 2150])

ro = gpd.sjoin(raion, oblast, predicate="within")
ro = ro.rename(columns={"geometry":"WKT"})[["oblast", "raion", "WKT"]]

# Разделяем Заполярный район НАО на три: восточный, западный и центральный.
# Потому что такое деление есть в файле охоты, и для данной области оно существенно.
rai = ro[ro.raion=="Заполярный район"]
rai_geo = shape(rai.iloc[0].WKT)

east_json = {
    "type":"Polygon",
    "coordinates":[[[66.52490242103188,69.68847622716733],[61.51239021400062,71.0879053921751],[56.28015144446941,71.08389618186156],
                    [56.51635749915683,69.74945572627142],[58.89489753821937,69.71041098650711],[59.58154304603192,69.44086193268213],
                    [59.62548835853185,68.9843019287944],[59.669433671031875,68.66855831699178],[59.82324226478193,68.34828715739798],
                    [61.635986405406925,66.77052037474171],[65.93164070228188,67.30678073621782],[66.52490242103188,69.68847622716733]]]
           
            }
west_json = {
    "type":"Polygon",
    "coordinates":[[[46.786243879969525,68.95198374496185],[40.470527629367545,68.92149227982124],[42.51273562608672,65.5765633144313],
                    [48.93848854468089,65.47687922599304],[52.570620764009334,66.40299178334469],[51.808221733337746,66.81583947154165],
                    [51.40722075677529,67.03566108997],[50.00097075677529,66.99056575219628],[49.83617583490025,67.05924901458843],
                    [49.968011772400246,67.17044495924509],[49.385736381775274,67.54511830991495],[49.13305083490026,67.48834459790652],
                    [48.671625053650274,67.56820931809462],[48.55077544427527,67.64780475406447],[47.553864427446456,68.42332825173933],
                    [46.786243879969525,68.95198374496185]]]
        }
east_mask = shape(east_json)
west_mask = shape(west_json)

rai_west = rai_geo.intersection(west_mask)
rai_east = rai_geo.intersection(east_mask)
rai_center = rai_geo.difference(west_mask).difference(east_mask)

ro = ro.drop(rai.index)
ro = ro.append({'oblast':'Ненецкий автономный округ', 'raion':'Заполярный район (центр)', 'WKT':str(rai_center)}, ignore_index=True)
ro = ro.append({'oblast':'Ненецкий автономный округ', 'raion':'Заполярный район (запад)', 'WKT':str(rai_west)}, ignore_index=True)
ro = ro.append({'oblast':'Ненецкий автономный округ', 'raion':'Заполярный район (восток)', 'WKT':str(rai_east)}, ignore_index=True)

ro.to_csv(os.path.join(ROOT, "joins/raion_oblast.csv"), index=False)

# Код пришивки страны, района и области к файлу птицы

In [7]:
class Country():
    def __init__(self):
        self.df = gpd.read_file(os.path.join(ROOT, "joins/countries_geojson.csv"), encoding="utf-8")
        self.df = self.df.loc[:, ["geojson", "NAME_RU"]].rename(columns={"NAME_RU":"country"})
        self.df["geometry"] = self.df.geojson.apply(lambda x: shape(json.loads(x)))

    def join_to_bird(self, bird:Bird):
        with LogOperation(LOG, bird, "join_country"):
            bird.df = gpd.sjoin(bird.df, self.df, how="left", predicate="within").loc[:,[*bird.df.columns,"country"]]

        LOG.counter_info(bird.name, "country", bird.df.country.value_counts(dropna=False).to_dict())


class RuRaionOblast():
    def __init__(self):
        self.df = gpd.read_file(os.path.join(ROOT, "joins/raion_oblast.csv"))

    def join_to_bird(self, bird:Bird):
        with LogOperation(LOG, bird, "join_raion"):
            bird.df = gpd.sjoin(bird.df, self.df, how="left", predicate="within").loc[:,[*bird.df.columns, "oblast", "raion"]]
            bird.df.loc[(bird.df.country=="Россия") & (bird.df.raion.isnull()), "country"] = np.nan
            bird.df.loc[~bird.df.raion.isnull(), "country"] = "Россия"

        LOG.counter_info(bird.name, "oblast", bird.df[bird.df.country=="Россия"].oblast.value_counts(dropna=False).to_dict())
        LOG.counter_info(bird.name, "raion", bird.df[bird.df.country=="Россия"][["oblast", "raion"]].value_counts(dropna=False).to_dict())
        LOG.counter_info(bird.name, "country_russia", bird.df.country.value_counts(dropna=False).to_dict())

# Цикл пришивки стран-областей-районов к птицам **02_clean -> 03_region**

In [36]:
%%time

SRC_DIR = os.path.join(ROOT, "results/02_clean")
SAVE_DIR = os.path.join(ROOT, "results/03_region")
LOG_DIR = os.path.join(ROOT, "logs")

bird_files = sorted(os.listdir(SRC_DIR))

disp = ProgressBox(stages={
        'geometry':'переводим в geopandas',
        'country':'пришиваем страны',
        'raion':'пришиваем районы и области'
    }, 
    total_files=len(bird_files))

!rm -r -f $SAVE_DIR/*.*

LOG = Logger(LOG_DIR, prefix="region_")

country = Country()
raion_oblast = RuRaionOblast()

for i, file in enumerate(bird_files):
    disp.new_file(file, count=i+1)
    bird = Bird(os.path.join(SRC_DIR, file))
    LOG.operation_info(bird.name, 'load', (0,0), bird.df.shape)

    disp.new_stage('geometry')
    bird.df = gpd.GeoDataFrame(bird.df, geometry=gpd.points_from_xy(bird.df.location_long, bird.df.location_lat))
    
    disp.new_stage('country')
    country.join_to_bird(bird)

    disp.new_stage('raion')
    raion_oblast.join_to_bird(bird)

    disp.new_stage("save")
    bird.df.drop(columns=['geometry'], inplace=True)
    bird.df[['country', 'oblast', 'raion']] = bird.df[['country', 'oblast', 'raion']].fillna('')
    bird.save(SAVE_DIR, "r")

    disp.new_stage("ok")

LOG.flush()

CPU times: user 59min 6s, sys: 42 s, total: 59min 48s
Wall time: 1h 54s


# Обработка логов пришивки областей и районов

In [37]:
LOG_DIR = os.path.join(ROOT, "logs")

# проверяем, было ли дублирование при мёрже стран-районов
# (когда одна запись в птицах пришилась одновременно к нескольким странам или областям-районам)
oper = pd.read_csv(os.path.join(LOG_DIR, "region_operations.csv"))
pi = oper.pivot(index="bird_file", columns="operation", values="shape_after").applymap(lambda sh:eval(sh)[0]).reset_index()
pi = pi[[pi.columns[0], pi.columns[3], pi.columns[1], pi.columns[2]]]
assert not any(pi.load!=pi.join_raion), "ВНИМАНИЕ! Есть дублирование при мёрже"
print("Всё ок, дублирования при мёрже не было")

Всё ок, дублирования при мёрже не было
